In [ ]:
from urllib.request import urlopen, Request
from urllib.error import HTTPError

from bs4 import BeautifulSoup
import os
import requests

import pandas as pd
import re
import pickle

import time

import numpy as np

from tqdm import trange, tqdm
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
def remove_text_between_parens(text):
    n = 1  # run at least once
    while n:
        text, n = re.subn(r'\<[^<>]*\>', '', text)  # remove non-nested/flat balanced parts
    return text

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# NY times

In [ ]:
years = list(range(2015, 2021))
months = [['01', '03'], ['04', '06'], ['07', '09'], ['10', '12']]

company = 'MasterCard'.lower()

titles = []
subtitles = []
links = []

with tqdm(desc="parsing", total= len(months) * len(years)) as pbar_outer:

    for year in years:
        for month in months:

            time.sleep(3)

            url = 'https://www.nytimes.com/search?dropmab=false&endDate={0}{2}01&query={3}&sort=best&startDate={0}{1}01'.format(str(year), month[0], month[1], company)
            #print(url)

            html = urlopen(url)

            s = str(html.read().decode('utf'))

            c = 0
            soup = BeautifulSoup(s)

            for p in soup.find_all('p', class_ = 'css-16nhkrn'):
                c+= 1
                #print(c)
                #print(remove_text_between_parens(str(div)))
                subtitles.append(remove_text_between_parens(str(p)))

            c = 0
            for h4 in soup.find_all('h4', class_ = 'css-2fgx4k'):
                c+= 1
                #print(c)
                #print(remove_text_between_parens(str(h4)))
                titles.append(remove_text_between_parens(str(h4)))

            c = 0
            for li in soup.find_all('li', class_ = 'css-1l4w6pd'):
                for a in li.find_all('a'):
                    #print(c)
                    #print(a.get('href'))
                    links.append(a.get('href'))
                    c+= 1 
            pbar_outer.update(1)


# Датафрейм

In [ ]:
amz = pd.DataFrame([titles, subtitles, links]).T
amz.columns = ['titles', 'subtitles', 'links']
amz

,titles,subtitles,links
0,Costco Customers May Come Out Ahead in Amex Di...,Severing an exclusive deal with the warehouse ...,https://dealbook.nytimes.com/2015/02/13/costco...
1,"Challenged by Upstarts, Lenders Try New Strate...","Rather than play defense, financial services f...",https://dealbook.nytimes.com/2015/01/07/challe...
2,Amex’s Loss in Court May Not Help Consumers,"If history is any guide, the ruling against Am...",/2015/02/24/business/dealbook/amexs-loss-in-co...
3,"American Express Violated Antitrust Laws, Judg...",The company has long barred merchants from enc...,/2015/02/20/business/american-express-violated...
4,Morning Agenda: Fed Shows Signs of Optimism,Fed officials are encouraged by the strength o...,https://dealbook.nytimes.com/2015/01/08/mornin...
...,...,...,...
198,The Media’s Complicated Relationship With Trump,International arrivals to New York are down as...,/2020/11/06/business/dealbook/media-trump-bump...
199,Why Dunkin’ Is Worth Nearly $9 Billion,"With business travel all but stopped, city hot...",/2020/10/26/business/dealbook/dunkin-inspire-d...
200,A Mighty Ant,None,/2020/10/27/business/dealbook/ant-group-IPO.ht...
201,"‘If No Tourists Come, I Have No Business’: New...",None,/2020/10/09/travel/nyc-tourism-travel-restrict...


In [ ]:
amz['dates'] = amz['links'].apply(lambda x: np.array(re.compile(r'\d{4}\/\d{2}\/\d{2}').findall(x)))
amz['dates_len'] = amz['dates'].apply(lambda x: len(x))
amz = amz[amz['dates_len'] != 0]
amz['news'] = amz['titles'] + amz['subtitles']
amz['dates'] = amz['dates'].apply(lambda x: x[0]).astype(str)
amz['dates'] = pd.to_datetime(amz['dates'])

amz

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,titles,subtitles,links,dates,dates_len,news
0,Costco Customers May Come Out Ahead in Amex Di...,Severing an exclusive deal with the warehouse ...,https://dealbook.nytimes.com/2015/02/13/costco...,2015-02-13,1,Costco Customers May Come Out Ahead in Amex Di...
1,"Challenged by Upstarts, Lenders Try New Strate...","Rather than play defense, financial services f...",https://dealbook.nytimes.com/2015/01/07/challe...,2015-01-07,1,"Challenged by Upstarts, Lenders Try New Strate..."
2,Amex’s Loss in Court May Not Help Consumers,"If history is any guide, the ruling against Am...",/2015/02/24/business/dealbook/amexs-loss-in-co...,2015-02-24,1,Amex’s Loss in Court May Not Help ConsumersIf ...
3,"American Express Violated Antitrust Laws, Judg...",The company has long barred merchants from enc...,/2015/02/20/business/american-express-violated...,2015-02-20,1,"American Express Violated Antitrust Laws, Judg..."
4,Morning Agenda: Fed Shows Signs of Optimism,Fed officials are encouraged by the strength o...,https://dealbook.nytimes.com/2015/01/08/mornin...,2015-01-08,1,Morning Agenda: Fed Shows Signs of OptimismFed...
...,...,...,...,...,...,...
198,The Media’s Complicated Relationship With Trump,International arrivals to New York are down as...,/2020/11/06/business/dealbook/media-trump-bump...,2020-11-06,1,The Media’s Complicated Relationship With Trum...
199,Why Dunkin’ Is Worth Nearly $9 Billion,"With business travel all but stopped, city hot...",/2020/10/26/business/dealbook/dunkin-inspire-d...,2020-10-26,1,Why Dunkin’ Is Worth Nearly $9 BillionWith bus...
200,A Mighty Ant,None,/2020/10/27/business/dealbook/ant-group-IPO.ht...,2020-10-27,1,NaN
201,"‘If No Tourists Come, I Have No Business’: New...",None,/2020/10/09/travel/nyc-tourism-travel-restrict...,2020-10-09,1,NaN


In [ ]:
amz = amz[['news', 'dates']]
print(amz.shape)
amz.head()

(195, 2)


,news,dates
0,Costco Customers May Come Out Ahead in Amex Di...,2015-02-13
1,"Challenged by Upstarts, Lenders Try New Strate...",2015-01-07
2,Amex’s Loss in Court May Not Help ConsumersIf ...,2015-02-24
3,"American Express Violated Antitrust Laws, Judg...",2015-02-20
4,Morning Agenda: Fed Shows Signs of OptimismFed...,2015-01-08


In [ ]:
amz = amz.dropna()
print(amz.shape)
amz.head()

(192, 2)


,news,dates
0,Costco Customers May Come Out Ahead in Amex Di...,2015-02-13
1,"Challenged by Upstarts, Lenders Try New Strate...",2015-01-07
2,Amex’s Loss in Court May Not Help ConsumersIf ...,2015-02-24
3,"American Express Violated Antitrust Laws, Judg...",2015-02-20
4,Morning Agenda: Fed Shows Signs of OptimismFed...,2015-01-08


In [ ]:
amz.to_csv('/content/drive/MyDrive/full stack/{}_parsed_news.csv'.format(company))

# Единый цикл с тиккерами

In [ ]:
[['01', '01'], ['04', '06'], ['07', '09'], ['10', '12']]

[['01', '01'], ['04', '06'], ['07', '09'], ['10', '12']]

In [ ]:
def parse(comp_name, comp_name_for_parse):
  years = list(range(2015, 2021))
  months = [['01', '01'],
            ['02', '02'],
            ['03', '03'],
            ['04', '04'],
            ['05', '05'],
            ['06', '06'],
            ['07', '07'],
            ['08', '08'],
            ['09', '09'],
            ['10', '10'],
            ['11', '11'],
            ['12', '12']]

  titles = []
  subtitles = []
  links = []
  #print(comp_name_for_parse)

  with tqdm(desc=comp_name, total= len(months) * len(years)) as pbar_outer:

      for year in years:
          for month in months:

              time.sleep(3)


              url = 'https://www.nytimes.com/search?dropmab=false&endDate={0}{2}28&query={3}&sections=Business%7Cnyt%3A%2F%2Fsection%2F0415b2b0-513a-5e78-80da-21ab770cb753&sort=best&startDate={0}{1}01'.format(str(year), month[0], month[1], comp_name_for_parse)
              #print(url)

              try: 
                html = urlopen(url)

                s = str(html.read().decode('utf'))

                c = 0
                soup = BeautifulSoup(s)

                for p in soup.find_all('p', class_ = 'css-16nhkrn'):
                    c+= 1
                    #print(c)
                    #print(remove_text_between_parens(str(div)))
                    subtitles.append(remove_text_between_parens(str(p)))

                c = 0
                for h4 in soup.find_all('h4', class_ = 'css-2fgx4k'):
                    c+= 1
                    #print(c)
                    #print(remove_text_between_parens(str(h4)))
                    titles.append(remove_text_between_parens(str(h4)))

                c = 0
                for li in soup.find_all('li', class_ = 'css-1l4w6pd'):
                    for a in li.find_all('a'):
                        #print(c)
                        #print(a.get('href'))
                        links.append(a.get('href'))
                        c+= 1 
                pbar_outer.update(1)

              except HTTPError:
                print('ERROR: {0} no data for {1}.{2}'.format(comp_name, month, year))
                continue
                pbar_outer.update(1)
                


  df = pd.DataFrame([titles, subtitles, links]).T
  df.columns = ['titles', 'subtitles', 'links']
  
  return df

In [ ]:
inp = """1. Newmont Goldcorp / NEM
2. Virgin Galactics / SPCE (Поразвлекаться можно, от новостей ее волатилит)
3. Microstrategy / MSTR
4. Transocean / RIG
5. Momo / MOMO
6. Beyond Meat / BYND
7. Zynerba Pharmaceuticals / ZYNE
8. Carnival Corporation / CCL
9. Etsy / ETSY
10. Lumentum / LITE"""

inp1= """Alibaba / BABA
Microsoft / MSFT
Apple / AAPL
AMD / AMD
Intel / INTC
Tesla / TSLA"""

inp = inp + '\n' + inp1

inp = inp.split('\n')
res = []

for i in inp:
  res.append(i.split(' / '))

res = np.array(res).T

comp_names = res[0]

comp_names = [re.sub(r'\d+. ', '', a) for a in comp_names]
#print(comp_names)

tickers = res[1]
tickers = [re.sub(r' \(.*\)', '', a) for a in tickers]

print(comp_names)
print(tickers)

['Newmont Goldcorp', 'Virgin Galactics', 'Microstrategy', 'Transocean', 'Momo', 'Beyond Meat', 'Zynerba Pharmaceuticals', 'Carnival Corporation', 'Etsy', 'Lumentum', 'Alibaba', 'Microsoft', 'Apple', 'AMD', 'Intel', 'Tesla']
['NEM', 'SPCE', 'MSTR', 'RIG', 'MOMO', 'BYND', 'ZYNE', 'CCL', 'ETSY', 'LITE', 'BABA', 'MSFT', 'AAPL', 'AMD', 'INTC', 'TSLA']


In [ ]:
tickers = ['AAPL', 'BABA', 'INTC', 'MSFT', 'TSLA', 'AMZN', 'T','CSCO','MU','NVDA','F','KO','AMD']
comp_names = ['Apple', 'Alibaba', 'Intel','Microsoft', 'Tesla', 'Amazon', 'AT&T', 'Cisco', 'Micron Technology', 'Nvidia', 'Ford', 'Coca-Cola', 'AMD']

In [ ]:
tickers = ['GOOGL', 'FB', 'EBAY', 'LRCX', 'NFLX', 'NKE', 'DIS','MCD','PYPL','SBUX','SMSNl','MA']
comp_names = ['google', 'facebook', 'ebay', 'lam research', 'netflix', 'nike', 'disney', 'McDonalds', 'paypal', 'starbucks', 'samsung', 'mastercard']

In [ ]:
len(tickers), len(comp_names)

(12, 12)

In [ ]:
name_tick_dict = dict(zip(tickers, comp_names))
name_tick_dict 

{'DIS': 'disney',
 'EBAY': 'ebay',
 'FB': 'facebook',
 'GOOGL': 'google',
 'LRCX': 'lam research',
 'MA': 'mastercard',
 'MCD': 'McDonalds',
 'NFLX': 'netflix',
 'NKE': 'nike',
 'PYPL': 'paypal',
 'SBUX': 'starbucks',
 'SMSNl': 'samsung'}

In [ ]:
dict_parses = {}

tickers = list(name_tick_dict.keys())

#for i in range(0, len(comp_names)):

for ticker in tickers:

    comp_name = name_tick_dict[ticker]
    ticker = ticker

    comp_name_for_parse = comp_name.replace(' ', '%20').replace('&', '%26')

    #print(ticker, )
    df_loc = parse(comp_name, comp_name_for_parse) 

    df_loc['dates'] = df_loc['links'].apply(lambda x: np.array(re.compile(r'\d{4}\/\d{2}\/\d{2}').findall(x)))
    df_loc['dates_len'] = df_loc['dates'].apply(lambda x: len(x))
    df_loc = df_loc[df_loc['dates_len'] != 0]
    df_loc['news'] = df_loc['titles'] + df_loc['subtitles']
    df_loc['dates'] = df_loc['dates'].apply(lambda x: x[0]).astype(str)
    df_loc['dates'] = pd.to_datetime(df_loc['dates'])

    df_loc = df_loc[['news', 'dates']]
    df_loc = df_loc.dropna()

    df_loc.to_csv('/content/drive/MyDrive/full stack/parse_data_monthes/{}_parsed_news.csv'.format(comp_name))

    #dict_parses[ticker] = df_loc

    with open('/content/drive/MyDrive/full stack/parse_data_monthes/parses_dict.pickle', 'rb') as f:
      dict_parses = pickle.load(f)

    dict_parses[ticker] = df_loc

    with open('/content/drive/MyDrive/full stack/parse_data_monthes/parses_dict.pickle', 'wb') as f:
      pickle.dump(dict_parses, f)
    


ERROR: facebook no data for ['10', '10'].2016



ERROR: ebay no data for ['12', '12'].2019



ERROR: lam research no data for ['07', '07'].2019



ERROR: netflix no data for ['04', '04'].2018
ERROR: netflix no data for ['12', '12'].2019



ERROR: nike no data for ['01', '01'].2016



ERROR: paypal no data for ['01', '01'].2017



In [ ]:
with open('/content/drive/MyDrive/full stack/parse_data_monthes/parses_dict.pickle', 'wb') as f:
  pickle.dump(dict_parses, f)

In [ ]:
dict_parses

{'AAPL':                                                   news      dates
 0    S.E.C. Has Yet to Set Rule on Tricky Ratio of ... 2015-01-26
 1    New York Attorney General Seeks Expanded Repor... 2015-01-14
 2    Amazon’s Tax Deal With Luxembourg May Break Ru... 2015-01-17
 3    How ‘1989’ Edged Out ‘Frozen’ as the No. 1 Alb... 2015-01-05
 4    Digital Tax Increase to Take Effect in EuropeN... 2015-01-02
 ..                                                 ...        ...
 699  What Did Google Do?The electric-car maker will... 2020-10-21
 700  Apple Puts Key Contractor on Probation Over La... 2020-11-09
 701  Apple will pay $113 million to settle states’ ... 2020-11-18
 702  Catch up on the latest news, from the DealBook... 2020-11-18
 703  Amazon Charged With Antitrust Violations by Eu... 2020-11-10
 
 [692 rows x 2 columns]}

In [ ]:
with open('/content/drive/MyDrive/full stack/parse_data/parses_dict.pickle', 'wb') as f:
  pickle.dump(dict_parses, f)

In [ ]:
with open('/content/drive/MyDrive/full stack/parse_data/parses_dict.pickle', 'rb') as f:
  dict_parses = pickle.load(f)